# Preparation

<b>Load Libraries</b>

In [5]:
# data structures
import numpy as np
import pandas as pd

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
## settings
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_theme('notebook')

<b>Load Dataset</b>

In [6]:
# dataset
df_base = pd.read_csv('../../dataset/cleaned/train.csv')

<b>Functions</b>

In [7]:
# graph: histogram
def plot_hist(ax, name: str) -> None:
    g = sns.histplot(
        data=df_base, 
        x=name, 
        hue='churn', 
        multiple='stack',
        kde=True,
        ax=ax
    )
    g.axes.lines[0].set_color('r'); g.axes.lines[1].set_color('r')
    g.set_ylabel(None)

In [8]:
# graph: boxen
def plot_boxen(ax, name: str) -> None:
    g = sns.boxenplot(
        data=df_base, 
        x=name, 
        hue='churn',
        ax=ax
    )

In [9]:
# graph: qq
def plot_qq(ax, name: str) -> None:
    churn = ['no', 'yes']

    for i in range(2):
        mask = df_base['churn'] == churn[i]
        data = df_base[mask]
        pp = sm.ProbPlot(
            data=data[name], 
            fit=True
        )
        g = pp.qqplot(
            marker='o', markerfacecolor=sns.color_palette()[i], markeredgecolor='none',
            alpha=0.5, 
            ax=ax
        )
        sm.qqline(
            ax=g, 
            line='45',
            fmt='r'
        )